In [12]:
%matplotlib inline
from IPython.display import HTML
import matplotlib.animation
import matplotlib.pyplot as plt

import autograd.numpy as np
from autograd import *

In [22]:
EARTH_GRAVITY_CONSTANT = 9.81  # m/s^2
l1 = 2.0
l2 = 1.0
mass = 0.42

f = lambda q: np.array([
        l1 * np.sin(q[0]),
        - l1 * np.cos(q[0]),
        l1 * np.sin(q[0]) + l2 * np.sin(q[0] + q[1]),
        - l1 * np.cos(q[0]) - l2 * np.cos(q[0] + q[1]),
    ])
M = mass * np.eye(4)
U = lambda q: mass * EARTH_GRAVITY_CONSTANT * f(q)[1]

J_f = jacobian(f)
H_f = hessian(f)
grad_U = grad(U)
K = lambda q: J_f(q).T @ M @ J_f(q)

q_dot = lambda p, q: np.linalg.inv(K(q)) @ p
p_dot = lambda p, q: p.T @ np.linalg.inv(K(q)) @ H_f(q).T @ M @ J_f(q) @ np.linalg.inv(K(q)) @ p - grad_U(q)

dt = 0.1
qs = [np.asarray([1.0, 0.0])]
ps = [K(qs[0]) @ np.asarray([0.0, 0.0])]
ts = [0]

for i in range(100):
    dq = q_dot(ps[-1], qs[-1])
    qs.append(qs[-1] + dt * dq)

    dp = p_dot(ps[-1], qs[-1])
    ps.append(ps[-1] + dt * dp)
    
    ts.append(ts[-1] + dt)

X1 = np.asarray([f(q)[0] for q in qs])
Y1 = np.asarray([f(q)[1] for q in qs])
X2 = np.asarray([f(q)[2] for q in qs])
Y2 = np.asarray([f(q)[3] for q in qs])

/home/salah/envs/arm-sim/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


In [23]:
%%capture

fig, ax = plt.subplots()
ax.axis([-(l1+l2), (l1+l2), -((l1+l2) + 0.5), 0.5])
ax.set_aspect('equal')
ax.set_xlabel('X[m]')
ax.set_ylabel('Y[m]')

link1, = ax.plot([], [], 'b-', lw=2)
link2, = ax.plot([], [], 'b-', lw=2)
c = ax.add_patch(plt.Circle((0, 0), 0.1))

def init():
    link1.set_data([], [])
    link2.set_data([], [])
    c.center = (X2[0], Y2[0])

def animate(i):
    link1.set_data([0, X1[i]], [0, Y1[i]])
    link2.set_data([X1[i], X2[i]], [Y1[i], Y2[i]])
    c.center = (X2[i], Y2[i])

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(X1), init_func=init)

In [24]:
HTML(ani.to_jshtml())